In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Shopee_comment_ranking
!ls

Mounted at /content/drive
/content/drive/MyDrive/Shopee_comment_ranking
'Bản sao của Chi Train PhoBert3.ipynb'	 teencode.xlsx
'Chi Train PhoBert2.ipynb'		 test-2.txt
'Chi Train PhoBert3.ipynb'		 test-3.txt
'Chi Train PhoBert5.ipynb'		 test-4.txt
 data1.csv				 test-5.txt
 Dataset_from_raw.ipynb			 test-6.txt
 new_data.csv				 test-7.txt
 PhoBERT_base_fairseq			 test-8.txt
 PhoBERT_base_fairseq.tar.gz		 test-9.txt
 PhoBERT_base_transformers		'Train PhoBert3.ipynb'
 PhoBERT_base_transformers.tar.gz	'Train PhoBert5.ipynb'
 pre_processing.ipynb			 transfer.ipynb
 shopee_rating.csv			 vncorenlp
 teencode.txt				 wordsegmenter.rdr.1


##Install requirements packages

In [ ]:
!pip install transformers
!pip install fastBPE
!pip install fairseq
# Install the vncorenlp python wrapper
!pip install vncorenlp
# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 43.6 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
     |████████████████████████████████| 6.6 MB 52.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp37-cp37m-linux_x86_64.whl size=483870 sha256=daaf9b1b7587b4c770e73b6c52ff332ba57ba930af349413e71743eccc5930f8
  Stored in directory: /root/.cache/pip/wheels/bd/d4/0e/0d317a65f77d3f8049fedd8a2ee0519164cf3e6bd77ef886f1
Successfully built fastBPE
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |█████████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 5.3 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=82398d3d905dafe393848935bc653730179c68632f942beb734a2496db91b52d
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp
--2022-06-06 19:12:38--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  62.1MB/s    in 0.4s    

2022-06

**Install VnCoreNLP**

In [ ]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/drive/MyDrive/Shopee_comment_ranking/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

**Install pre-trained PhoBERT**

In [ ]:
!wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -xzvf PhoBERT_base_transformers.tar.gz

--2022-06-06 19:12:45--  https://public.vinai.io/PhoBERT_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 18.160.200.87, 18.160.200.105, 18.160.200.53, ...
Connecting to public.vinai.io (public.vinai.io)|18.160.200.87|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322405979 (307M) [application/x-tar]
Saving to: ‘PhoBERT_base_transformers.tar.gz.1’

PhoBERT_base_transf 100%[===================>] 307.47M  11.6MB/s    in 14s     

2022-06-06 19:13:02 (21.6 MB/s) - ‘PhoBERT_base_transformers.tar.gz.1’ saved [322405979/322405979]

PhoBERT_base_transformers/
PhoBERT_base_transformers/config.json
PhoBERT_base_transformers/bpe.codes
PhoBERT_base_transformers/model.bin
PhoBERT_base_transformers/dict.txt


In [ ]:
!wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
!tar -xzvf PhoBERT_base_fairseq.tar.gz

--2022-06-06 19:13:12--  https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
Resolving public.vinai.io (public.vinai.io)... 18.160.200.87, 18.160.200.105, 18.160.200.53, ...
Connecting to public.vinai.io (public.vinai.io)|18.160.200.87|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1243308020 (1.2G) [application/x-tar]
Saving to: ‘PhoBERT_base_fairseq.tar.gz.1’

PhoBERT_base_fairse 100%[===================>]   1.16G  63.7MB/s    in 20s     

2022-06-06 19:13:33 (58.5 MB/s) - ‘PhoBERT_base_fairseq.tar.gz.1’ saved [1243308020/1243308020]

PhoBERT_base_fairseq/
PhoBERT_base_fairseq/bpe.codes
PhoBERT_base_fairseq/model.pt
PhoBERT_base_fairseq/dict.txt


**Load Phobert**

In [ ]:
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes', 
    default="/content/drive/MyDrive/Shopee_comment_ranking/PhoBERT_base_transformers/bpe.codes",
    required=False,
    type=str,
    help='path to fastBPE BPE'
)
args, unknown = parser.parse_known_args()
bpe = fastBPE(args)

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file("/content/drive/MyDrive/Shopee_comment_ranking/PhoBERT_base_transformers/dict.txt")

# Data preparing

In [ ]:
import re

train_path = '/content/drive/MyDrive/Shopee_comment_ranking/test-9.txt'
test_path = '/content/drive/MyDrive/Shopee_comment_ranking/test-2.txt'

train_id, train_text, train_labels = [], [], []
test_id, test_text = [], []

with open(train_path, 'r') as f_r:
    data = f_r.read().strip()

    data = re.findall('train_[\s\S]+?\n.*\n[1-2]\n\n', data)

    for sample in data:
        splits = sample.strip().split('\n')

        id = splits[0]
        label = int(splits[-1])
        text = ' '.join(splits[1:-1])
        text = rdrsegmenter.tokenize(text)
        text = ' '.join([' '.join(x) for x in text])

        train_id.append(id)
        train_text.append(text)
        train_labels.append(label)


with open(test_path, 'r') as f_r:
    data = f_r.read().strip()
    data = re.findall('test_[\s\S]+?\n.*\n\n\n', data)

    for sample in data:
        splits = sample.strip().split('\n')

        id = splits[0]
        text = ' '.join(splits[1:])
        text = rdrsegmenter.tokenize(text)
        text = ' '.join([' '.join(x) for x in text])

        test_id.append(id)
        test_text.append(text)
for i in range(len(train_labels)):
  train_labels[i] -= 1

**Train test split**

In [ ]:
from sklearn.model_selection import train_test_split

train_sents, val_sents, train_labels, val_labels = train_test_split(train_text, train_labels, test_size=0.1)

In [ ]:
# train_labels

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LEN = 125

train_ids = []
for sent in train_sents:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    train_ids.append(encoded_sent)

val_ids = []
for sent in val_sents:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    val_ids.append(encoded_sent)
    
train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
val_ids = pad_sequences(val_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

In [ ]:
train_masks = []
for sent in train_ids:
    mask = [int(token_id > 0) for token_id in sent]
    train_masks.append(mask)

val_masks = []
for sent in val_ids:
    mask = [int(token_id > 0) for token_id in sent]

    val_masks.append(mask)

**DataLoader**

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
train_inputs = torch.tensor(train_ids)
val_inputs = torch.tensor(val_ids)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=64)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=64)

# Train

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

config = RobertaConfig.from_pretrained(
    "/content/drive/MyDrive/Shopee_comment_ranking/PhoBERT_base_transformers/config.json", from_tf=False, num_labels = 2, output_hidden_states=False,
)
BERT_SA = RobertaForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/Shopee_comment_ranking/PhoBERT_base_transformers/model.bin",
    config=config
)

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at /content/drive/MyDrive/Shopee_comment_ranking/PhoBERT_base_transformers/model.bin were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequen

In [ ]:
BERT_SA.cuda()
print('Done')

Done


In [ ]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    F1_score = f1_score(pred_flat, labels_flat, average='macro')
    
    return accuracy_score(pred_flat, labels_flat), F1_score

In [ ]:
def model_phobert(lr1, decay):
  epochs = 5
  max = 0

  param_optimizer = list(BERT_SA.named_parameters())
  no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': decay},
      {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': decay}
  ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=lr1, correct_bias=False)


  for epoch_i in range(0, epochs):
      print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      print('Training...')

      total_loss = 0
      BERT_SA.train()
      train_accuracy = 0
      nb_train_steps = 0
      train_f1 = 0
      
      for step, batch in tqdm_notebook(enumerate(train_dataloader)):
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)

          BERT_SA.zero_grad()
          outputs = BERT_SA(b_input_ids, 
              token_type_ids=None, 
              attention_mask=b_input_mask, 
              labels=b_labels)
          loss = outputs[0]
          total_loss += loss.item()
          
          logits = outputs[1].detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()
          tmp_train_accuracy, tmp_train_f1 = flat_accuracy(logits, label_ids)
          train_accuracy += tmp_train_accuracy
          train_f1 += tmp_train_f1
          nb_train_steps += 1
          
          loss.backward()
          torch.nn.utils.clip_grad_norm_(BERT_SA.parameters(), 1.0)
          optimizer.step()
          
      avg_train_loss = total_loss / len(train_dataloader)
      print(" Accuracy: {0:.4f}".format(train_accuracy/nb_train_steps))
      print(" F1 score: {0:.4f}".format(train_f1/nb_train_steps))
      print(" Average training loss: {0:.4f}".format(avg_train_loss))
      BERT_SA.save_pretrained('/content/drive/MyDrive/Shopee_comment_ranking/{}'.format(epoch_i+1))
      print("Running Validation...")
      BERT_SA.eval()
      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0
      eval_f1 = 0
      for batch in tqdm_notebook(val_dataloader):

          batch = tuple(t.to(device) for t in batch)

          b_input_ids, b_input_mask, b_labels = batch

          with torch.no_grad():
              outputs = BERT_SA(b_input_ids, 
              token_type_ids=None, 
              attention_mask=b_input_mask)
              logits = outputs[0]
              logits = logits.detach().cpu().numpy()
              label_ids = b_labels.to('cpu').numpy()

              tmp_eval_accuracy, tmp_eval_f1 = flat_accuracy(logits, label_ids)

              eval_accuracy += tmp_eval_accuracy
              eval_f1 += tmp_eval_f1
              nb_eval_steps += 1
      if max < eval_accuracy/nb_eval_steps:
        max = eval_accuracy/nb_eval_steps
        e = epoch_i
      print('logits:    {}'.format(np.argmax(logits, axis=1).flatten()))
      print('labels_id: {}'.format(label_ids))
      print(" Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
      print(" F1 score: {0:.4f}".format(eval_f1/nb_eval_steps))
  print("Training complete!")
  print('Best Valid acc : {}, epoch: {}'.format(max, e+1))

In [ ]:
import random
from tqdm import tqdm_notebook
device = 'cuda'

In [ ]:
model_phobert(5e-5,0.01)

======== Epoch 1 / 5 ========
Training...


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

 Accuracy: 0.7598
 F1 score: 0.7129
 Average training loss: 0.5085
Running Validation...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

0it [00:00, ?it/s]

 Accuracy: 0.9206
 F1 score: 0.9151
 Average training loss: 0.2598
Running Validation...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/9 [00:00<?, ?it/s]

logits:    [0 1 0 1 1 1 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 0 1 0
 0 0 0 0 1 0 0 0 0 1]
labels_id: [0 1 0 1 1 1 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 0 1 1 0
 0 1 1 0 1 0 1 0 1 1]
 Accuracy: 0.8370
 F1 score: 0.8289
Training complete!
Best Valid acc : 0.8369902482269503, epoch: 5


In [ ]:
model_phobert(1e-5,0.01)

======== Epoch 1 / 5 ========
Training...


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

 Accuracy: 0.9400
 F1 score: 0.9364
 Average training loss: 0.2036
Running Validation...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

0it [00:00, ?it/s]

 Accuracy: 0.9647
 F1 score: 0.9628
 Average training loss: 0.1350
Running Validation...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/9 [00:00<?, ?it/s]

logits:    [0 1 0 1 1 1 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 0 1 0
 0 0 0 0 1 0 1 0 1 1]
labels_id: [0 1 0 1 1 1 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 0 1 1 0
 0 1 1 0 1 0 1 0 1 1]
 Accuracy: 0.8348
 F1 score: 0.8300
Training complete!
Best Valid acc : 0.8480348699763594, epoch: 1


In [ ]:
model_phobert(5e-6,0.01)

======== Epoch 1 / 5 ========
Training...


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

 Accuracy: 0.9648
 F1 score: 0.9627
 Average training loss: 0.1302
Running Validation...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

0it [00:00, ?it/s]

 Accuracy: 0.9750
 F1 score: 0.9737
 Average training loss: 0.1001
Running Validation...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/9 [00:00<?, ?it/s]

logits:    [0 1 0 1 1 1 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 0 1 0
 0 0 0 0 1 0 0 0 1 1]
labels_id: [0 1 0 1 1 1 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 0 1 1 0
 0 1 1 0 1 0 1 0 1 1]
 Accuracy: 0.8365
 F1 score: 0.8291
Training complete!
Best Valid acc : 0.8417183806146572, epoch: 2


In [ ]:
model_phobert(1e-6,0.01)

======== Epoch 1 / 5 ========
Training...


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

 Accuracy: 0.9752
 F1 score: 0.9738
 Average training loss: 0.0976
Running Validation...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

0it [00:00, ?it/s]

 Accuracy: 0.9759
 F1 score: 0.9746
 Average training loss: 0.0958
Running Validation...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/9 [00:00<?, ?it/s]

logits:    [0 1 0 1 0 1 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 0 1 0
 0 0 0 0 1 0 0 0 1 1]
labels_id: [0 1 0 1 1 1 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 0 1 1 0
 0 1 1 0 1 0 1 0 1 1]
 Accuracy: 0.8376
 F1 score: 0.8302
Training complete!
Best Valid acc : 0.8428265366430261, epoch: 4


In [ ]:
model_phobert(5e-6,0.01)

======== Epoch 1 / 5 ========
Training...


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

 Accuracy: 0.9760
 F1 score: 0.9748
 Average training loss: 0.0979
Running Validation...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

0it [00:00, ?it/s]

 Accuracy: 0.9790
 F1 score: 0.9780
 Average training loss: 0.0844
Running Validation...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/9 [00:00<?, ?it/s]

logits:    [0 1 0 1 1 1 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 0 1 0
 0 0 0 0 1 0 0 0 1 1]
labels_id: [0 1 0 1 1 1 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 0 1 1 0
 0 1 1 0 1 0 1 0 1 1]
 Accuracy: 0.8376
 F1 score: 0.8306
Training complete!
Best Valid acc : 0.8376182033096927, epoch: 5


In [ ]:
model_phobert(5e-6,0.01)

======== Epoch 1 / 5 ========
Training...


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

 Accuracy: 0.9815
 F1 score: 0.9806
 Average training loss: 0.0736
Running Validation...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

0it [00:00, ?it/s]

 Accuracy: 0.9850
 F1 score: 0.9843
 Average training loss: 0.0604
Running Validation...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/9 [00:00<?, ?it/s]

logits:    [0 1 0 1 1 1 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 0 1 0
 0 0 0 0 1 0 0 0 1 1]
labels_id: [0 1 0 1 1 1 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 0 1 1 0
 0 1 1 0 1 0 1 0 1 1]
 Accuracy: 0.8376
 F1 score: 0.8300
Training complete!
Best Valid acc : 0.8421985815602837, epoch: 4
